## Projeto 3 - Felipe Aron, Eduardo Vaz e Pedro Lopes

### Bitcoin price

#### Introdução

Desde a ascêndencia da criptografia e da computação no final do século XX, as inovações digitais têm modificado a maneira como os índividuos se interagem e como é transmitida a informação. Em 2008, após muito desenvolvimento programacional de seus antecedentes Wei Dai, Adam Back, entre outros, Satoshi Nakamoto publicou o whitepaper do Bitcoin, que viria a se tornar o primeiro passo para a construção de um novo sistema de transações de valor. Foi criada uma ferramenta de armazenamento de dados capaz de construir um sistema descentralizado de validação de transações e requerimento de dados de forma completamente "trustless". Deu-se o nome para essa tecnologia de blockchain, pela forma como é escrita ao longo do tempo.



Começou-se uma nova revolução moedas virtuais chegaram para instaurar um novo modelo de negócio e de transação financeira. Sendo assim, buscamos nesse documento, analisar o movimento de valor de mercado do Bitcoin. Essa análise, vai se basear em buscar e comparar outros movimentos de mercado que possam influenciar o Bitcoin. 

Por exemplo, movimento de outros mercados financeiros e suas bolsas de valores, como São Paulo, Tokyo, Nova York e Xangai por exemplo. Outra variável que pode influenciar é a quantidade de carteiras, que seriam como “contas” contendo bitcoin no mundo todo, ela pode de acordo com certa análise representar a oferta e demanda do bitcoin.

Vamos utilizar o dataset que contem o valor da bitcoin em função do tempo, e analisar nesse mesmo tempo, movimento de outras bolsas e outros mercados pra analisar a reação da criptomoeda. Como o mercado da bitcoin não fecha, não abertura e fechamento de bolsa, consequentemente.

In [1]:
import pandas as pd
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import linregress
import statsmodels.api as sm

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Dudu Vaz\Desktop


In [3]:
data_wallets = pd.read_csv('MyWallet.csv',sep=',')
data_price = pd.read_csv('MyPrice.csv',sep=',')
data_transactions = pd.read_csv('MyTransactions.csv',sep=',')
data_difficult = pd.read_csv('MyDifficult.csv',sep=',')

FileNotFoundError: File b'MyWallet.csv' does not exist

### Adjusting dataframe

In [ ]:
# Renomeamos a coluna do número de wallet's ativas em um dia para "Number of active BTC wallets in the network"
data_wallets = data_wallets.rename(columns={'Value':"Number of active BTC wallets in the network"})
data_wallets.head()

In [ ]:
# Renomeamos a coluna do preço do BTC em U$ para "BTC Price"
data_price = data_price.rename(columns={'Value':'BTC price'})
data_price.head()

In [ ]:
# Renomeamos a coluna do número de transações na rede BTC em um dia para "Number of transactions"
data_transactions = data_transactions.rename(columns={'Value':'Number of transactions'})
data_transactions.head()

In [ ]:
# Renomeamos a coluna da difficuldade de mineração de um bloco em um dia para "Minning difficult"
data_difficult = data_difficult.rename(columns={'Value':'Minning difficult'})
data_difficult.head()

In [ ]:
# Usando o metódo 'concat' juntamos os quatro dataframes ['data_wallets','data_price','data_transactions',data_difficult] em
# um único dataframe df_btc com as chaves 'Date' para a data, 'data_price' para o preço, 'data_wallets' para o número de wallets,
# 'data_transactions' para número de transações na rede em um dia e 'data_difficult' para a dificuldade de mineração de um bloco
df_btc = pd.concat([data_price['Date'],data_price['BTC price'],data_wallets['Number of active BTC wallets in the network'],\
                    data_transactions['Number of transactions'],data_difficult['Minning difficult']],\
                    axis=1, keys=['Date','data_price', 'data_wallets','data_transactions','data_difficult'])

In [ ]:
df_btc.describe()

In [ ]:
df_adjusted = df_btc[df_btc.data_price != 0] # ---> Cleaning dataframe wich has 0 values for BTC prices
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_adjusted)

In [ ]:
# Com o metódo 'assign' criamos uma nova coluna para o dataframe que calcula o delta do preço do BTC em U$ de um dia anterior 
# para o atual com o metódo 'diff'
df_btc_adjusted = df_adjusted.assign(DeltaBTCPrice=pd.Series(df_adjusted.data_price.diff().fillna(0).astype(float)).values)    

In [ ]:
df_btc_adjusted.DeltaBTCPrice.describe()

In [ ]:
# Com o metódo 'assign' criamos uma nova coluna para o dataframe que calcula o delta do número de wallets ativas na rede do BTC
df_btc_adjusted_wallets = df_btc_adjusted.assign(DeltaBTCWallets=pd.Series(df_btc_adjusted.data_wallets.diff().fillna(0).astype(float)).values)

In [ ]:
df_btc_adjusted_wallets.DeltaBTCWallets.describe() # ---> Number of unique bitcoin addresses used per day.

In [ ]:
# Com o metódo 'assign' criamos uma nova coluna para o dataframe que calcula o delta do número de transações na rede do BTC
df_btc_adjusted_transactions = df_btc_adjusted_wallets.assign(DeltaBTCTransactions=pd.Series(df_btc_adjusted_wallets.data_transactions.diff().fillna(0).astype(float)).values)

In [ ]:
df_btc_adjusted_transactions.DeltaBTCTransactions.describe()

In [ ]:
# Com o metódo 'assign' criamos uma nova coluna para o dataframe que calcula o delta da dificuldade de mineração
# de blocos na rede do BTC
df_btc_adjusted_difficult = df_btc_adjusted_transactions.assign(DeltaBTCDifficult=pd.Series(df_btc_adjusted_transactions.data_difficult.diff().fillna(0).astype(float)).values)

In [ ]:
df_btc_adjusted_difficult.DeltaBTCDifficult.describe()

In [ ]:
df = df_btc_adjusted_difficult.dropna(axis=0, how='any')
df

In [ ]:
#Separando o dataframe unido agora em base treino e base teste
comp = np.random.rand(len(df)) < 0.75

df = df[comp]

test = df[~comp]

df

## Modelos preditivos

Com o dataframe organizado e com um leque razoável de variáveis para análise realizamos as implementações dos modelos preditivos:

- Modelo de predição pela média

- Modelo dos K vizinhos mais próximos (K-Nearest Neighbors Regression)

- Modelo de Regressão Linear (Multiple Linear Regression)

- Modelo de árvores de regressão (Decision Tree Regression)

### Modelo de predição pela média

Usando somente a variável em Y para fazer essa predição que é a média de todos os valores para o preco do BTC entre 2010-08-17 e 2018-05-22 (Year-Month-Day)

In [ ]:
print("A média dps valores do preço do BTC em U$ é {0}".format(df.data_price.mean()))

### Modelo dos K vizinhos mais próximos

### Modelo de  Regressão linear 

In [ ]:
def lin_regression(x, y):
    m, b, R, p, SEm = linregress(x, y)

    n = len(x)
    SSx = np.var(x, ddof=1) * (n-1)  
    SEb2 = SEm **2 * (SSx/n + np.mean(x)**2)
    SEb = SEb2**0.5

    return m, b, SEm, SEb, R, p

In [ ]:
# Análise do coeficiente de determinação do preço do BTC em U$ em relação ao Delta do número de wallet's ativas na rede BTC

m, b, Sm, Sb, R, p = lin_regression(df.data_price,df.DeltaBTCWallets)

print('m = {:>.4g} +- {:6.4f}'.format(m, Sm))
print('b = {:>.4g} +- {:6.4f}\n'.format(b, Sb))

print('R2 = {:7.5f}'.format(R**2))
print('p of test F : {:<8.6f}'.format(p))

In [ ]:
Y = df.data_price
X = df.DeltaBTCWallets
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.rsquared

Como o resultado para o "R-squared" foi próximo a zero podemos pontuar que o Delta de Wallet's ativas na rede BTC não interfere no preço em U$ como tínhamos apontado como uma das hipóteses

In [ ]:
# Análise do coeficiente de determinação do preço do BTC em U$ em relação ao Delta do número de transações em um dia na rede BTC

m, b, Sm, Sb, R, p = lin_regression(df.data_price,df.DeltaBTCTransactions)

print('m = {:>.4g} +- {:6.4f}'.format(m, Sm))
print('b = {:>.4g} +- {:6.4f}\n'.format(b, Sb))

print('R2 = {:7.5f}'.format(R**2))
print('p of test F : {:<8.6f}'.format(p))

In [ ]:
Y = df.data_price
X = df.DeltaBTCTransactions
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.rsquared

Como o resultado para o "R-squared" foi próximo a zero podemos pontuar que o Delta de transações na rede BTC não interfere no preço em U$ como tínhamos apontado como uma das hipóteses.

In [ ]:
# Análise do coeficiente de determinação do preço do BTC em U$ em relação ao Delta da dificuldade de mineração de blocos
# na rede BTC

m, b, Sm, Sb, R, p = lin_regression(df.data_price,df.DeltaBTCDifficult)

print('m = {:>.4g} +- {:6.4f}'.format(m, Sm))
print('b = {:>.4g} +- {:6.4f}\n'.format(b, Sb))

print('R2 = {:7.5f}'.format(R**2))
print('p of test F : {:<8.6f}'.format(p))

In [ ]:
Y = df.data_price
X = df.DeltaBTCDifficult
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.rsquared

Como o resultado para o "R-squared" foi próximo a zero podemos pontuar que o Delta da dificuldade de mineração  de blocos na rede BTC não interfere no preço em U$ como tínhamos apontado como uma das hipóteses. Porém, esse havia sido o maior 'R-squared' atingido portanto fizemos a análise do coeficiente de determinação do valor em relação a dificuldade e chegamos à um alto 'R-squared'

In [ ]:
# Análise do coeficiente de determinação do preço do BTC em U$ em relação à dificuldade de mineração de blocos na rede BTC

m, b, Sm, Sb, R, p = lin_regression(df.data_price,df.data_difficult)

print('m = {:>.4g} +- {:6.4f}'.format(m, Sm))
print('b = {:>.4g} +- {:6.4f}\n'.format(b, Sb))

print('R2 = {:7.5f}'.format(R**2))
print('p of test F : {:<8.6f}'.format(p))

In [ ]:
Y = df.data_price
X = df.data_difficult
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.rsquared

In [ ]:
# Explicação price x difficult

In [ ]:
# Análise do coeficiente de determinação do preço do BTC em U$ em relação à wallet's ativas na rede BTC

m, b, Sm, Sb, R, p = lin_regression(df.data_price,df.data_wallets)

print('m = {:>.4g} +- {:6.4f}'.format(m, Sm))
print('b = {:>.4g} +- {:6.4f}\n'.format(b, Sb))

print('R2 = {:7.5f}'.format(R**2))
print('p of test F : {:<8.6f}'.format(p))

In [ ]:
Y = df.data_price
X = df.data_wallets
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.rsquared

In [ ]:
# Explicação price x wallets

In [ ]:
# Análise do coeficiente de determinação do preço do BTC em U$ em relação à transações na rede BTC

m, b, Sm, Sb, R, p = lin_regression(df.data_price,df.data_transactions)

print('m = {:>.4g} +- {:6.4f}'.format(m, Sm))
print('b = {:>.4g} +- {:6.4f}\n'.format(b, Sb))

print('R2 = {:7.5f}'.format(R**2))
print('p of test F : {:<8.6f}'.format(p))

In [ ]:
Y = df.data_price
X = df.data_transactions
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
results.rsquared

In [ ]:
#Analise do coeficiente de determinação do Preço de BTC em U$ em relação à transações na rede BTC para base teste
m, b, Sm, Sb, R, p = lin_regression(test.data_price,test.data_transactions)

print('m = {:>.4g} +- {:6.4f}'.format(m, Sm))
print('b = {:>.4g} +- {:6.4f}\n'.format(b, Sb))

print('R2 = {:7.5f}'.format(R**2))
print('p of test F : {:<8.6f}'.format(p))

A analise do coeficiente, utlizando dessa vez a base teste, nos apresenta um valor que na realidade é bem próximo ao da base treino. Isso acontece muito devido ao dataset ter sido dividido e apurado com muito cuidado, logo uma % dele, mesmo sendo menor, não consegue ter uma alteração tão grande no valor final, a diferença é pequena.

In [ ]:
# Explicação price x transactios

In [ ]:
df.plot.scatter('data_price','data_difficult')

In [ ]:
df.plot.scatter('data_price','data_wallets')

In [ ]:
df.plot.scatter('data_price','data_transactions')

### Modelo de árvores de regressão (Decision Tree Regression)